In [33]:
dataset.shape

(202599, 42)

In [42]:
data_skin = np.array(dataset['Pale_Skin'])
data_skin=np.maximum(data_skin, 0)
print(data_skin)
data_ = data_skin[np.where(data_skin==1)[0]]
print(data_.shape)

[0 0 0 ... 0 0 1]
(8701,)


In [43]:
import pandas as pd
import numpy as np
dataset = pd.read_csv("list_attr_celeba.txt", sep = ' ', header=1, skiprows = 0)

data = np.vstack((dataset['Male'], dataset["Young"], dataset["Mustache"], (-dataset["No_Beard"]),\
                                                                dataset["Bald"]))

# np.where(data > 0, data, 0)
data=np.maximum(data, 0)
print("0:Male, 1:Young, 2: Mustache, 3:Beard, 4:Bald")
print(data.sum(axis=1))

0:Male, 1:Young, 2: Mustache, 3:Beard, 4:Bald
[ 84434 156734   8417  33441   4547]


In [44]:
data_ = data[:,np.where(data[0,:]==0)[0]]
print(data_.shape)
data_1 = data_[:,np.where(data_[1,:]==0)[0]]
print(data_1.shape)
data_2 = data_1[:,np.where(data_1[2,:]==0)[0]]
print(data_2.shape)
data_3 = data_2[:,np.where(data_2[3,:]==0)[0]]
print(data_3.shape)
print(len(np.where(data_3[4,:]==0)[0]))

(5, 118165)
(5, 14878)
(5, 14876)
(5, 14835)
14831


In [45]:
data.shape

(5, 202599)

In [46]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

## Training

In [60]:


n_steps = int(data.shape[1]/1000)



# clear the param store in case we're in a REPL
pyro.clear_param_store()


def model(data):
    # sample f from the beta prior
    laten_sex = pyro.param('latent_sex', torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    
    laten_mus_1 = pyro.param("latent_female_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus_2 = pyro.param("latent_male_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus = [laten_mus_1,laten_mus_2]
    
    laten_beard_1 = pyro.param("latent_no_mustache_beard", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_beard_2 = pyro.param("latent_mustache_beard", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_beard = [laten_beard_1,laten_beard_2]

    
    laten_young = pyro.param('latent_young', torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    
    laten_bald_1 = pyro.param("latent_female_old_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_2 = pyro.param("latent_female_young_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_3 = pyro.param("latent_male_old_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_4 = pyro.param("latent_male_young_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald = [[laten_bald_1,laten_bald_2],[laten_bald_3, laten_bald_4]]
    
    
    # loop over the observed data
    for i in pyro.plate("data_loop", data.shape[1]):
        # observe datapoint i using the bernoulli likelihood
        sex = pyro.sample('sex_{}'.format(i), dist.Bernoulli(laten_sex),obs = data[0,i])        
        young = pyro.sample('young_{}'.format(i), dist.Bernoulli(laten_young),obs = data[1,i])        

        mus = pyro.sample("mustache_{}".format(i), dist.Bernoulli(laten_mus[sex.long()]), obs=data[2,i])
        beard = pyro.sample("beard_{}".format(i), dist.Bernoulli(laten_beard[mus.long()]), obs=data[3,i])
        
        
        bald = pyro.sample("bald_{}".format(i), dist.Bernoulli(laten_bald[sex.long()][young.long()]), \
                                                             obs=data[4,i])
        


def guide(data):
    pass

store_p = []                       
# setup the optimizer
adam_params = {"lr": 0.1, "betas": (0.80, 0.999)}
optimizer = Adam(adam_params)
pyro.clear_param_store()
# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# do gradient steps
for step in range(n_steps):
    rand_list = np.random.randint(low = 0, high=data.shape[1], size=1000)
#     svi.step(data[:,rand_list.tolist()])
    svi.step(torch.tensor(data[:,rand_list.tolist()]).float())
    if(step == 50):
        adam_params = {"lr": 0.01, "betas": (0.80, 0.999)}
        optimizer = Adam(adam_params)
    if(step == 100):
        adam_params = {"lr": 0.0005, "betas": (0.80, 0.999)}
        optimizer = Adam(adam_params)
    if(step %1 ==0):
        print('step: {}'.format(step))
#         for i in pyro.params.param_store.keys():
#             store_p[i] = pyro.params(i)
        p = pyro.get_param_store()
        store_p.append(p) 
        for (k,v) in p.items():
            print(k+": {:.3f}".format(v.item())) 




step: 0
latent_sex: 0.475
latent_female_mustache: 0.475
latent_male_mustache: 0.475
latent_no_mustache_beard: 0.475
latent_mustache_beard: 0.525
latent_young: 0.525
latent_female_old_bald: 0.475
latent_female_young_bald: 0.475
latent_male_old_bald: 0.475
latent_male_young_bald: 0.475
step: 1
latent_sex: 0.451
latent_female_mustache: 0.450
latent_male_mustache: 0.450
latent_no_mustache_beard: 0.450
latent_mustache_beard: 0.550
latent_young: 0.550
latent_female_old_bald: 0.450
latent_female_young_bald: 0.450
latent_male_old_bald: 0.450
latent_male_young_bald: 0.450
step: 2
latent_sex: 0.431
latent_female_mustache: 0.426
latent_male_mustache: 0.426
latent_no_mustache_beard: 0.426
latent_mustache_beard: 0.574
latent_young: 0.574
latent_female_old_bald: 0.425
latent_female_young_bald: 0.426
latent_male_old_bald: 0.426
latent_male_young_bald: 0.426
step: 3
latent_sex: 0.415
latent_female_mustache: 0.402
latent_male_mustache: 0.402
latent_no_mustache_beard: 0.402
latent_mustache_beard: 0.597


step: 29
latent_sex: 0.424
latent_female_mustache: 0.095
latent_male_mustache: 0.128
latent_no_mustache_beard: 0.137
latent_mustache_beard: 0.898
latent_young: 0.785
latent_female_old_bald: 0.094
latent_female_young_bald: 0.095
latent_male_old_bald: 0.139
latent_male_young_bald: 0.098
step: 30
latent_sex: 0.426
latent_female_mustache: 0.091
latent_male_mustache: 0.125
latent_no_mustache_beard: 0.136
latent_mustache_beard: 0.902
latent_young: 0.784
latent_female_old_bald: 0.091
latent_female_young_bald: 0.091
latent_male_old_bald: 0.136
latent_male_young_bald: 0.095
step: 31
latent_sex: 0.424
latent_female_mustache: 0.087
latent_male_mustache: 0.123
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.905
latent_young: 0.783
latent_female_old_bald: 0.087
latent_female_young_bald: 0.087
latent_male_old_bald: 0.133
latent_male_young_bald: 0.091
step: 32
latent_sex: 0.421
latent_female_mustache: 0.084
latent_male_mustache: 0.120
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.

step: 58
latent_sex: 0.423
latent_female_mustache: 0.040
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.130
latent_mustache_beard: 0.946
latent_young: 0.775
latent_female_old_bald: 0.040
latent_female_young_bald: 0.040
latent_male_old_bald: 0.113
latent_male_young_bald: 0.047
step: 59
latent_sex: 0.426
latent_female_mustache: 0.039
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.130
latent_mustache_beard: 0.947
latent_young: 0.775
latent_female_old_bald: 0.039
latent_female_young_bald: 0.039
latent_male_old_bald: 0.113
latent_male_young_bald: 0.047
step: 60
latent_sex: 0.434
latent_female_mustache: 0.039
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.131
latent_mustache_beard: 0.949
latent_young: 0.775
latent_female_old_bald: 0.038
latent_female_young_bald: 0.039
latent_male_old_bald: 0.112
latent_male_young_bald: 0.046
step: 61
latent_sex: 0.433
latent_female_mustache: 0.038
latent_male_mustache: 0.099
latent_no_mustache_beard: 0.131
latent_mustache_beard: 0.

step: 87
latent_sex: 0.427
latent_female_mustache: 0.024
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.127
latent_mustache_beard: 0.963
latent_young: 0.773
latent_female_old_bald: 0.024
latent_female_young_bald: 0.024
latent_male_old_bald: 0.109
latent_male_young_bald: 0.032
step: 88
latent_sex: 0.428
latent_female_mustache: 0.023
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.127
latent_mustache_beard: 0.963
latent_young: 0.773
latent_female_old_bald: 0.023
latent_female_young_bald: 0.023
latent_male_old_bald: 0.108
latent_male_young_bald: 0.032
step: 89
latent_sex: 0.424
latent_female_mustache: 0.023
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.128
latent_mustache_beard: 0.963
latent_young: 0.776
latent_female_old_bald: 0.023
latent_female_young_bald: 0.023
latent_male_old_bald: 0.108
latent_male_young_bald: 0.031
step: 90
latent_sex: 0.415
latent_female_mustache: 0.023
latent_male_mustache: 0.097
latent_no_mustache_beard: 0.128
latent_mustache_beard: 0.

step: 116
latent_sex: 0.420
latent_female_mustache: 0.016
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.124
latent_mustache_beard: 0.965
latent_young: 0.771
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.120
latent_male_young_bald: 0.025
step: 117
latent_sex: 0.420
latent_female_mustache: 0.016
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.125
latent_mustache_beard: 0.965
latent_young: 0.769
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.120
latent_male_young_bald: 0.025
step: 118
latent_sex: 0.424
latent_female_mustache: 0.016
latent_male_mustache: 0.101
latent_no_mustache_beard: 0.125
latent_mustache_beard: 0.965
latent_young: 0.768
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.121
latent_male_young_bald: 0.024
step: 119
latent_sex: 0.420
latent_female_mustache: 0.015
latent_male_mustache: 0.101
latent_no_mustache_beard: 0.125
latent_mustache_beard

step: 145
latent_sex: 0.419
latent_female_mustache: 0.012
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.132
latent_mustache_beard: 0.972
latent_young: 0.771
latent_female_old_bald: 0.012
latent_female_young_bald: 0.012
latent_male_old_bald: 0.111
latent_male_young_bald: 0.024
step: 146
latent_sex: 0.413
latent_female_mustache: 0.012
latent_male_mustache: 0.099
latent_no_mustache_beard: 0.132
latent_mustache_beard: 0.972
latent_young: 0.769
latent_female_old_bald: 0.012
latent_female_young_bald: 0.012
latent_male_old_bald: 0.110
latent_male_young_bald: 0.024
step: 147
latent_sex: 0.406
latent_female_mustache: 0.011
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.132
latent_mustache_beard: 0.972
latent_young: 0.770
latent_female_old_bald: 0.012
latent_female_young_bald: 0.012
latent_male_old_bald: 0.110
latent_male_young_bald: 0.024
step: 148
latent_sex: 0.405
latent_female_mustache: 0.011
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.131
latent_mustache_beard

step: 174
latent_sex: 0.406
latent_female_mustache: 0.009
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.130
latent_mustache_beard: 0.973
latent_young: 0.772
latent_female_old_bald: 0.009
latent_female_young_bald: 0.009
latent_male_old_bald: 0.108
latent_male_young_bald: 0.022
step: 175
latent_sex: 0.404
latent_female_mustache: 0.009
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.130
latent_mustache_beard: 0.974
latent_young: 0.772
latent_female_old_bald: 0.009
latent_female_young_bald: 0.009
latent_male_old_bald: 0.109
latent_male_young_bald: 0.022
step: 176
latent_sex: 0.404
latent_female_mustache: 0.009
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.131
latent_mustache_beard: 0.973
latent_young: 0.772
latent_female_old_bald: 0.009
latent_female_young_bald: 0.009
latent_male_old_bald: 0.111
latent_male_young_bald: 0.022
step: 177
latent_sex: 0.411
latent_female_mustache: 0.009
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.131
latent_mustache_beard

In [61]:

all_param = pyro.get_param_store()
print(all_param)
all_param.save('./pyro_params')

## Testing

In [62]:
graph_params = pyro.params.param_store.ParamStoreDict()
graph_params.load('./pyro_params')

In [63]:
for (k,v) in graph_params.items():
    print(k+": {:.3f}".format(v.item())) 

latent_sex: 0.402
latent_female_mustache: 0.007
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.133
latent_mustache_beard: 0.971
latent_young: 0.776
latent_female_old_bald: 0.008
latent_female_young_bald: 0.007
latent_male_old_bald: 0.121
latent_male_young_bald: 0.021


In [64]:
from pyro import poutine
from pyro.infer import config_enumerate, infer_discrete

def determined_model():
    laten_sex = graph_params.get_param('latent_sex')

    laten_mus_1 = graph_params.get_param("latent_female_mustache")
    laten_mus_2 = graph_params.get_param("latent_male_mustache")
    laten_mus = [laten_mus_1, laten_mus_2]

    laten_beard_1 = graph_params.get_param("latent_no_mustache_beard")
    laten_beard_2 = graph_params.get_param("latent_mustache_beard")
    laten_beard = [laten_beard_1, laten_beard_2]

    laten_young = graph_params.get_param('latent_young')

    laten_bald_1 = graph_params.get_param("latent_female_old_bald")
    laten_bald_2 = graph_params.get_param("latent_female_young_bald")
    laten_bald_3 = graph_params.get_param("latent_male_old_bald")
    laten_bald_4 = graph_params.get_param("latent_male_young_bald")
    laten_bald = [[laten_bald_1, laten_bald_2], [laten_bald_3, laten_bald_4]]

    with pyro.plate("a_plate", size=1, dim=-2):
        sex = pyro.sample('sex', dist.Bernoulli(laten_sex))
        young = pyro.sample('young', dist.Bernoulli(laten_young))
        #         with pyro.plate("b_plate", size=1, dim=-1):

        mus = pyro.sample("mustache", dist.Bernoulli(laten_mus[sex.long()]))
        beard = pyro.sample("beard", dist.Bernoulli(laten_beard[mus.long()]))

        bald = pyro.sample("bald", dist.Bernoulli(laten_bald[sex.long()][young.long()]))

        
def make_log_joint(model):

    def _log_joint(data, *args, **kwargs):

        conditioned_model = poutine.condition(model, data=data)

        trace = poutine.trace(conditioned_model).get_trace(*args, **kwargs)

        return trace.log_prob_sum()

    return _log_joint

 

scale_log_joint = make_log_joint(determined_model)

print(scale_log_joint({\
                       "sex": torch.tensor(0.0),
                       "young": torch.tensor(0.),
                       "mustache": torch.tensor(0.0),
                       "beard": torch.tensor(.0),
                       "bald":torch.tensor(.0)\
}).exp())


tensor(0.1143, grad_fn=<ExpBackward>)


### Compute joint probability learned by Pyro

In [65]:
from itertools import product

In [66]:
perm_array = np.array(list(product([0, 1], repeat=5))).astype(float)

def joint_prob_graph(arr):
    prob = scale_log_joint({\
                       "sex": torch.tensor(arr[0]),
                       "young": torch.tensor(arr[1]),
                       "mustache": torch.tensor(arr[2]),
                       "beard": torch.tensor(arr[3]),
                       "bald":torch.tensor(arr[4])\
                           }).exp()
    return prob.data.numpy()

In [67]:
perm_array

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0.],
       [0., 0., 1., 1., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 1., 0., 1.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 1.],
       [1., 0., 1., 1., 0.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 0., 0.],
       [1., 1., 0., 0., 1.],
       [1., 1., 0., 1., 0.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1.]])

In [68]:
learned_joint_prob_array = []

for arr in perm_array:
    p = joint_prob_graph(arr)
    learned_joint_prob_array.append(p)

learned_joint_prob_array = np.array(learned_joint_prob_array)
learned_joint_prob_array

array([1.14329420e-01, 8.71653105e-04, 1.75486776e-02, 1.33791979e-04,
       2.77513555e-05, 2.11577695e-07, 9.42341962e-04, 7.18446135e-06,
       3.96290158e-01, 2.93919126e-03, 6.08274600e-02, 4.51143020e-04,
       9.61921180e-05, 7.13434404e-07, 3.26635827e-03, 2.42258140e-05,
       6.17586492e-02, 8.53769816e-03, 9.47947277e-03, 1.31047033e-03,
       2.25985611e-04, 3.12409186e-05, 7.67370531e-03, 1.06083570e-03,
       2.38604020e-01, 5.00798401e-03, 3.66238630e-02, 7.68686632e-04,
       8.73093497e-04, 1.83250822e-05, 2.96472956e-02, 6.22257675e-04])

### Joint probability derived from dataset 

In [69]:
ori_joint_prob_array = []

for arr in perm_array:
    p = ((data.T == arr).sum(axis=1) == 5).mean()
    ori_joint_prob_array.append(p)

ori_joint_prob_array = np.array(ori_joint_prob_array)
ori_joint_prob_array

array([7.32037177e-02, 1.97434341e-05, 2.02370199e-04, 0.00000000e+00,
       4.93585852e-06, 0.00000000e+00, 4.93585852e-06, 0.00000000e+00,
       5.09262139e-01, 6.41661607e-05, 4.78778276e-04, 0.00000000e+00,
       4.93585852e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.62985503e-02, 1.04344049e-02, 3.10317425e-02, 4.33861964e-03,
       5.57752013e-04, 6.91020193e-05, 1.78332568e-02, 2.38401966e-03,
       1.52745078e-01, 1.75222977e-03, 8.60764367e-02, 2.54690300e-03,
       4.83714135e-04, 3.94868681e-05, 1.93683088e-02, 7.94673221e-04])

In [70]:
np.mean((ori_joint_prob_array - learned_joint_prob_array)**2)**0.5

0.030382503760488736

### Compare the conditioned probability from the generated dataset

In [94]:
res = np.load("./1211_use_pyro.npy")

att_name = np.array(["Bald",
"Bangs",
"Black_Hair",
"Blond_Hair",
"Brown_Hair",
"Bushy_Eyebrows",
"Eyeglasses",
"Male",
"Mouth_Slightly_Open",
"Mustache",
"No_Beard",
"Pale_Skin",
"Young"])
att_name_list = np.array([7,12,9,10,0])
print(att_name[att_name_list])

res = res.squeeze()
data = []
for i in range(att_name_list.shape[0]):
    if(att_name[att_name_list[i]] == "No_Beard"):
        data.append(-1*res[:,i])
    else:
        data.append(res[:,i])
data = np.array(data).astype(int)
data = np.maximum(data, 0)
print(data)

perm_array = np.array(list(product([0, 1], repeat=5)))
generated_joint_prob_array = []

for arr in perm_array:
    p = ((data.T == arr).sum(axis=1) == 5).mean()
    generated_joint_prob_array.append(p)

generated_joint_prob_array = np.array(generated_joint_prob_array)
generated_joint_prob_array

['Male' 'Young' 'Mustache' 'No_Beard' 'Bald']
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]]


array([0.101, 0.005, 0.367, 0.116, 0.   , 0.   , 0.205, 0.003, 0.025,
       0.   , 0.037, 0.035, 0.   , 0.   , 0.024, 0.001, 0.003, 0.   ,
       0.077, 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   ])

In [87]:
def prob_edited_attr(prob_a):
    return prob_a[np.where(np.logical_and(perm_array[:,1] == 1 , perm_array[:,2] == 1))[0]]
prob_edited_attr(generated_joint_prob_array).sum()

0.025

In [79]:
generated_conditoned_prob_array = prob_edited_attr(generated_joint_prob_array) / prob_edited_attr(generated_joint_prob_array).sum()
ori_conditoned_prob_array = prob_edited_attr(ori_joint_prob_array) / prob_edited_attr(ori_joint_prob_array).sum()

In [80]:
generated_conditoned_prob_array

array([0.11150235, 0.00469484, 0.43544601, 0.1842723 , 0.        ,
       0.        , 0.2629108 , 0.00117371])

In [81]:
ori_conditoned_prob_array

array([9.96840973e-01, 2.68853341e-04, 2.75574674e-03, 0.00000000e+00,
       6.72133351e-05, 0.00000000e+00, 6.72133351e-05, 0.00000000e+00])

In [82]:
np.mean((generated_conditoned_prob_array - ori_conditoned_prob_array)**2)**0.5

0.3664200710923157